In [154]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [160]:
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [161]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station)
assembly = celeri.create_assembly_dictionary()
assembly = celeri.merge_geodetic_data(assembly, station, sar) 
assembly, block_constraint_partials = celeri.block_constraints(assembly, block, command)

In [205]:
def get_cross_partials(vector):
    """
    Returns a linear operator R that when multiplied by 
    vector a gives the cross product a cross b
    """
    return np.array([[0, vector[2], -vector[1]], [-vector[2], 0, vector[0]], [vector[1], -vector[0], 0]])


def cartesian_vector_to_spherical_vector(vel_x, vel_y, vel_z, lon, lat):
    """
    This function transforms vectors from Cartesian to spherical components.
    Arguments:
        vel_x: array of x components of velocity
        vel_y: array of y components of velocity
        vel_z: array of z components of velocity
        lon: array of station longitudes
        lat: array of station latitudes
    Returned variables:
        vel_north: array of north components of velocity
        vel_east: array of east components of velocity
        vel_up: array of up components of velocity
    """
    projection_matrix = np.array([[-np.sin(np.deg2rad(lat)) * np.cos(np.deg2rad(lon)), -np.sin(np.deg2rad(lat)) * np.sin(np.deg2rad(lon)), np.cos(np.deg2rad(lat))],
                                  [-np.sin(np.deg2rad(lon)), np.cos(np.deg2rad(lon)), 0],
                                  [-np.cos(np.deg2rad(lat)) * np.cos(np.deg2rad(lon)), -np.cos(np.deg2rad(lat)) * np.sin(np.deg2rad(lon)), -np.sin(np.deg2rad(lat))]])
    vel_north, vel_east, vel_up = np.dot(projection_matrix, np.array([vel_x, vel_y, vel_z]))
    return vel_north, vel_east, vel_up


def get_fault_slip_rate_partials(segment, block):
    """
    Calculate partial derivatives for slip rate constraints
    """
    n_segments = len(segment)
    n_blocks = len(block)
    fault_slip_rate_partials = np.zeros((3 * n_segments, 3 * n_blocks))
    for i in range(n_segments):
        # Project velocities from Cartesian to spherical coordinates at segment mid-points
        row_idx = 3 * i
        column_idx_east = 3 * segment.east_labels[i]
        column_idx_west = 3 * segment.west_labels[i]
        R = get_cross_partials([segment.mid_x[i], segment.mid_y[i], segment.mid_z[i]])
        vn_wx, ve_wx, vu_wx = cartesian_vector_to_spherical_vector(R[0, 0], R[1, 0], R[2, 0], segment.mid_lon[i], segment.mid_lat[i])
        vn_wy, ve_wy, vu_wy = cartesian_vector_to_spherical_vector(R[0, 1], R[1, 1], R[2, 1], segment.mid_lon[i], segment.mid_lat[i])
        vn_wz, ve_wz, vu_wz = cartesian_vector_to_spherical_vector(R[0, 2], R[1, 2], R[2, 2], segment.mid_lon[i], segment.mid_lat[i])

        # Build unit vector for the fault
        # Projection on to fault strike
        # segment_azimuth = azimuth(segment.lat1[i], segment.lon1[i], segment.lat2[i], segment.lon2[i])
        segment_azimuth, _, _ = celeri.GEOID.inv( segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i])  # TODO: Need to check this vs. matlab azimuth for consistency
        uxpar = np.cos(np.deg2rad(90 - segment_azimuth))
        uypar = np.sin(np.deg2rad(90 - segment_azimuth))
        uxper = np.sin(np.deg2rad(segment_azimuth - 90))
        uyper = np.cos(np.deg2rad(segment_azimuth - 90))
        
        # Projection onto fault dip
        if segment.lat2[i] < segment.lat1[i]:
            uxpar = -uxpar
            uypar = -uypar
            uxper = -uxper
            uyper = -uyper

        # This is the logic for dipping vs. non-dipping faults
        # TODO: Better document the logic here
        if segment.dip[i] != 90:
            sf = 1/abs(np.cos(np.deg2rad(segment.dip[i])))
            R = np.array([[uxpar * ve_wx + uypar * vn_wx, uxpar * ve_wy + uypar * vn_wy, uxpar * ve_wz + uypar * vn_wz],
                          [sf * (uxper * ve_wx + uyper * vn_wx), sf * (uxper * ve_wy + uyper * vn_wy), sf * (uxper * ve_wz + uyper * vn_wz)],
                          [0, 0, 0]])
        else:
            sf = -1
            R = np.array([[uxpar * ve_wx + uypar * vn_wx, uxpar * ve_wy + uypar * vn_wy, uxpar * ve_wz + uypar * vn_wz],
                          [0, 0, 0],
                          [sf * (uxper * ve_wx + uyper * vn_wx), sf * (uxper * ve_wy + uyper * vn_wy), sf * (uxper * ve_wz + uyper * vn_wz)]])

        fault_slip_rate_partials[row_idx:row_idx + 3, column_idx_east:column_idx_east + 3] = R
        fault_slip_rate_partials[row_idx:row_idx + 3, column_idx_west:column_idx_west + 3] = -R
    return fault_slip_rate_partials


# function [slipCon, Index, Data, Sigma] = SlipConstraints(Segment, Block, Index, Data, Sigma, Command);
print("Isolating slip rate constraints")
# for i in range(len(segment.lon1)):
#     if segment.ss_rate_flag[i] == 1:
#         "{:.2f}".format(segment.ss_rate[i])
#         print("- Strike-slip constraint on " + segment.name[i].strip() + ": rate = " + "{:.2f}".format(segment.ss_rate[i]) + " (mm/yr), 1-sigma uncertainty = +/-" + "{:.2f}".format(segment.ss_rate_sig[i]) + " (mm/yr)")
#     if segment.ds_rate_flag[i] == 1:
#         print("- Dip-slip constraint on " + segment.name[i].strip() + ": rate = " + "{:.2f}".format(segment.ds_rate[i]) + " (mm/yr), 1-sigma uncertainty = +/-" + "{:.2f}".format(segment.ds_rate_sig[i]) + " (mm/yr)")
#     if segment.ts_rate_flag[i] == 1:
#         print("- Tensile-slip constraint on " + segment.name[i].strip() + ": rate = " + "{:.2f}".format(segment.ts_rate[i]) + " (mm/yr), 1-sigma uncertainty = +/-" + "{:.2f}".format(segment.ts_rate_sig[i]) + " (mm/yr)")

slip_constraints = get_fault_slip_rate_partials(segment, block)
# slipToggle = stack3([Segment.ssRateTog(:) Segment.dsRateTog(:) Segment.tsRateTog(:)]);
# Index.slip_constraints = find(slipToggle)
# Data.nSlipCon = length(Index.slipCon);
# Data.slipCon = stack3([Segment.ssRate(:) Segment.dsRate(:) Segment.tsRate(:)]);
# Data.slipCon = Data.slipCon(Index.slipCon);
# Sigma.slipCon = stack3([Segment.ssRateSig(:) Segment.dsRateSig(:) Segment.tsRateSig(:)]);
# Sigma.slipCon = Sigma.slipCon(Index.slipCon);
# slipCon = slipCon(Index.slipCon, :);
assembly["sigma"]["slip_constraint_weight"] = command["slip_constraint_weight"]


Isolating slip rate constraints


In [200]:
a = 1
b = np.array([1])
c = np.array(1)

In [213]:
len(segment)

837